In [15]:
import pickle
from heapq import heappop, heappush
from sliding_puzzle import SlidingBoard, Pattern, load_pdb, random_board
import copy

In [106]:
with open("./random_puzzle_1000.pkl", 'rb') as file_handle:
    data = pickle.load(file_handle)


In [87]:
def manhattan_dist(state: Pattern, goal = None):
    if goal is None:
        goal = SlidingBoard(L=state.L)
    cnt = 0
    for p in state.pos:
        # print( f"{p}: {state.pos[p]},  {goal.pos[p]}, {abs(state.pos[p][0] - goal.pos[p][0]) + abs(state.pos[p][1] - goal.pos[p][1])}")
        cnt += sum(map(abs, [i[0] - i[1] for i in zip(state.pos[p], goal.pos[p])]))
    return cnt

In [88]:
sum([manhattan_dist(i) for i in data]) / len(data)

20.02

In [89]:
data[0]

[4, 6, 0, 8]
[12, 7, 2, 1]
[9, 14, 10, 3]
[5, 13, 15, 11]

In [90]:
manhattan_dist(data[0])

26

In [91]:
h1 = load_pdb("./15_puzzle_12_13_14_addidtive_pdb.pkl")
h2 = load_pdb("./15_puzzle_1_2_4_5_8_9_addidtive_pdb.pkl")
h3 = load_pdb("./15_puzzle_3_6_7_10_11_15_addidtive_pdb.pkl")

In [92]:
def additive_heuristic(state: Pattern):
    sym = sym_board(state)
    return max(h1[state] + h2[state] + h3[state], h1[sym] + h2[sym] + h3[sym])

In [93]:
symetry(4)

1

In [94]:
def symetry(idx, L=4):
    return (idx % L) * L +  int(idx / L)

In [95]:
def sym_board(state):
    pos = {}
    for k in state.pos:
        pos[symetry(k)] = (state.pos[k][1], state.pos[k][0])
    return SlidingBoard(pos=pos)
    

In [96]:
sym_board(data[0])

[1, 3, 6, 5]
[9, 13, 11, 7]
[0, 8, 10, 15]
[2, 4, 12, 14]

In [97]:
sum([additive_heuristic(i) for i in data]) / len(data)

22.26

In [98]:
additive_heuristic(data[25])

24

In [99]:
def search(start, goal, heuristic):
    closed_set = set()
    gscore = {start:0}
    came_from = {}
    oheap = []

    heappush(oheap, (gscore[start] + heuristic(start), start))

    while(oheap):
        current = heappop(oheap)[1]
        if current in closed_set:
            continue
        if current == goal:
            path = []
            # print(f"path_length: {gscore[current]}")
            while current in came_from:
                path.append(current)
                current = came_from[current]
            
            path.append(start)
            path.reverse()
            
            return path

        closed_set.add(current)

        for neighbor in current.get_children():
            # allow reopen
            if neighbor in closed_set:
                continue

            if neighbor not in gscore or gscore[current] + 1 < gscore[neighbor]:
                gscore[neighbor] = gscore[current] + 1
                came_from[neighbor] = current
                heappush(oheap, (gscore[neighbor] + heuristic(neighbor) , neighbor))



In [100]:
data[0]

[4, 6, 0, 8]
[12, 7, 2, 1]
[9, 14, 10, 3]
[5, 13, 15, 11]

In [101]:
manhattan_dist(SlidingBoard().get_children()[0].get_children()[0], SlidingBoard())

2

In [104]:
search(data[0], SlidingBoard(), heuristic=manhattan_dist)

[[4, 6, 0, 8]
 [12, 7, 2, 1]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 8, 0]
 [12, 7, 2, 1]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 8, 1]
 [12, 7, 2, 0]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 8, 1]
 [12, 7, 0, 2]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 0, 1]
 [12, 7, 8, 2]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 1, 0]
 [12, 7, 8, 2]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 1, 2]
 [12, 7, 8, 0]
 [9, 14, 10, 3]
 [5, 13, 15, 11], [4, 6, 1, 2]
 [12, 7, 8, 3]
 [9, 14, 10, 0]
 [5, 13, 15, 11], [4, 6, 1, 2]
 [12, 7, 8, 3]
 [9, 14, 10, 11]
 [5, 13, 15, 0], [4, 6, 1, 2]
 [12, 7, 8, 3]
 [9, 14, 10, 11]
 [5, 13, 0, 15], [4, 6, 1, 2]
 [12, 7, 8, 3]
 [9, 14, 0, 11]
 [5, 13, 10, 15], [4, 6, 1, 2]
 [12, 7, 0, 3]
 [9, 14, 8, 11]
 [5, 13, 10, 15], [4, 6, 1, 2]
 [12, 0, 7, 3]
 [9, 14, 8, 11]
 [5, 13, 10, 15], [4, 0, 1, 2]
 [12, 6, 7, 3]
 [9, 14, 8, 11]
 [5, 13, 10, 15], [4, 1, 0, 2]
 [12, 6, 7, 3]
 [9, 14, 8, 11]
 [5, 13, 10, 15], [4, 1, 2, 0]
 [12, 6, 7, 3]
 [9, 14, 8, 11]
 [5, 13, 10, 15], [4, 1, 

In [107]:
search(data[0], SlidingBoard(), heuristic=additive_heuristic)

KeyboardInterrupt: 